In [10]:
import io
import nltk
import string
import warnings
import pdfminer
import pandas as pd
from pdfminer.layout import LAParams
from pdfminer.high_level import extract_text
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity


In [11]:
def extract_keywords(text):
    # Tokenización de palabras y eliminación de palabras vacías y signos de puntuación
    stop_words = set(stopwords.words('english') + list(string.punctuation))
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.lower() not in stop_words and word.isalpha()]

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

def extract_keywords(text):
    # Creación de la matriz de características usando TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([text])

    # Extracción de las palabras clave con mayor TF-IDF score
    feature_names = tfidf_vectorizer.vocabulary_
    sorted_feature_names = sorted(feature_names, key=lambda x: feature_names[x], reverse=True)
    keyword_indices = [list(feature_names.values()).index(feature_names[f]) for f in sorted_feature_names[:10]]
    keywords = [sorted_feature_names[i] for i in keyword_indices]

    return keywords

text = "Este es un ejemplo de texto para crear la matriz de características usando TF-IDF"
keywords = extract_keywords(text)
print(keywords)

['de', 'tf', 'crear', 'matriz', 'la', 'es', 'este', 'características', 'usando', 'un']


In [13]:
def pdf_to_text(file_path):
    output_string = io.StringIO()
    with open(file_path, 'rb') as in_file:
        rsrcmgr = PDFResourceManager()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, output_string, codec=codec, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.get_pages(in_file):
            interpreter.process_page(page)
        device.close()
    text = output_string.getvalue()
    output_string.close()
    return text

In [21]:
# Ejemplo de uso
pdf_file = 'CV_Kalash.pdf'
text = pdf_to_text(pdf_file)
keywords = extract_keywords(text)
print(keywords)


['playing', 'works', 'upto', 'date', 'keras', 'around', 'giving', 'create', 'got', 'placed']


In [27]:
# Función para preprocesar texto
def preprocess_text(text):
    # Convertir todo a minúsculas
    text = text.lower()
    # Eliminar puntuación
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Eliminar stopwords
    stop_words = set(stopwords.words("English"))
    words = text.split()
    words = [word for word in words if word not in stop_words]
    # Unir palabras de nuevo en una sola cadena
    text = " ".join(words)
    return text

In [29]:
# Extraer texto del PDF
pdf_path = "CV_Kalash.pdf"
pdf_text = extract_text(pdf_path)

In [30]:
# Leer archivo CSV con oferta de empleo
csv_path = "output.csv"
df = pd.read_csv(csv_path)

In [31]:
# Preprocesar texto de ambas fuentes de datos
pdf_text = preprocess_text(pdf_text)
csv_text = preprocess_text(" ".join(df.iloc[:, 0].tolist()))

In [32]:
# Calcular similitud coseno
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform([pdf_text, csv_text])
similarity_score = cosine_similarity(tfidf)[0, 1]

In [33]:
print("La similitud entre el texto del PDF y el archivo CSV es:", similarity_score)

La similitud entre el texto del PDF y el archivo CSV es: 0.09793218795014397
